# Yolo8云端部署
#### 2025.2.17
## 过程说明:
1. 分别载入模型和数据集
2. 整体迁移至工程文件夹下
3. 运行训练脚本
4. [可选]运行推理，结果保存在/kaggle/working/pred_rst.jpg文件
5. 转存训练好的权重文件至/kaggle/working/train.zip
## 注意:
1. 开启Kaggle平台的GPU（P100）后，运行nvidia --smi 可以看到该GPU支持CUDA12.6版本。而且note环境里已经预装了CUDA11.2、torch2.5.1+cu121

In [1]:
# 工程准备
print("复制工程到根目录")
!cp -r /kaggle/input/yolo8/pytorch/1.0/1/yolo8 /yolo8
!ls /yolo8 -a
!pip install -e /yolo8/
# 数据集准备
!cp -r /kaggle/input/river-floating-trash-datasets/datasets /yolo8/datasets
!ls /yolo8 -a

# 查看GPU是否存在：最高支持CUDA12.6
import torch
print("[0]GPU:")
def check_gpu_availability():
    # 使用 torch.cuda.is_available() 函数检测 GPU 是否可用
    if torch.cuda.is_available():
        # 获取可用 GPU 的数量
        num_gpus = torch.cuda.device_count()
        print(f"GPU 可用，可用的 GPU 数量为: {num_gpus}")
        # 遍历每个可用的 GPU 并打印其名称
        for i in range(num_gpus):
            gpu_name = torch.cuda.get_device_name(i)
            print(f"GPU {i}: {gpu_name}")
    else:
        print("GPU 不可用，将使用 CPU 进行计算。")
check_gpu_availability()
# 查看CUDA版本12.2
print("[1]CUDA:")
!nvcc --version
# 查看pytorch版本2.5.1
print("[2]torch:"+torch.__version__)

# 查看python版本3.10
print("[3]Python:")
!python --version
# 测试已通过

复制工程到根目录
.		 examples     none.txt		       setup.py
..		 .git	      .pre-commit-config.yaml  task
CITATION.cff	 .gitignore   README.md		       tests
CONTRIBUTING.md  LICENSE      README.zh-CN.md	       ultralytics
docker		 MANIFEST.in  requirements.txt	       ultralytics.egg-info
docs		 mkdocs.yml   setup.cfg		       yolov8n.pt
Obtaining file:///yolo8
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ultralytics
.		 examples     .pre-commit-config.yaml  tests
..		 .git	      README.md		       ultralytics
CITATION.cff	 .gitignore   README.zh-CN.md	       ultralytics.egg-info
CONTRIBUTING.md  LICENSE      requirements.txt	       yolov8n.pt
datasets	 MANIFEST.in  setup.cfg
docker		 mkdocs.yml   setup.py
docs		 none.txt     task
[0]GPU:
GPU 可用，可用的 GPU 数量为: 1
GPU 0: Tesla P100-PCIE-16GB
[1]CUDA:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cud

In [2]:
ls /yolo8 -a

./               examples/    .pre-commit-config.yaml  tests/
../              .git/        README.md                ultralytics/
CITATION.cff     .gitignore   README.zh-CN.md          ultralytics.egg-info/
CONTRIBUTING.md  LICENSE      requirements.txt         yolov8n.pt
datasets/        MANIFEST.in  setup.cfg
docker/          mkdocs.yml   setup.py
docs/            none.txt     task/


In [3]:
cd /yolo8

/yolo8


In [4]:
# 开始训练 或 推理

# 所有相对路径基于/yolo8


# 0训练    1推理
way = 0


def train():
    from ultralytics import YOLO
    # Load a model
    model = YOLO("yolov8n.yaml")  # build a new model from scratch
    #model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

    # Use the model
    results = model.train(data="./datasets/RFT.yaml", epochs=1,imgsz=416,device=0)  # train the model

def pred(image_path,rst_path):
    from ultralytics import YOLO
    import cv2
    import yaml

    # 加载训练好的权重
    model = YOLO('./runs/detect/train/weights/best.pt')
    results = model.predict(source=image_path)

    # 读取原始图像
    image = cv2.imread(image_path)
    cv2.imwrite('/kaggle/working/pred_src.jpg',image)

    # 处理推理结果
    for result in results:
        boxes = result.boxes  # 获取边界框信息
        if boxes is not None:
            for box in boxes:
                # 获取边界框坐标
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                x1, y1, x2, y2 = xyxy

                # 获取类别索引和置信度
                cls = int(box.cls[0])
                conf = float(box.conf[0])

                with open('./datasets/RFT.yaml', 'r') as f:
                    data = yaml.safe_load(f)
                names = data['names']

                # 绘制边界框
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # 准备标签文本
                label = f'{names[cls]}: {conf:.2f}'

                # 绘制标签背景
                (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(image, (x1, y1 - label_height - 5), (x1 + label_width, y1), (0, 255, 0), -1)

                # 绘制标签文本
                cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # 保存带有检测结果的图像
    return cv2.imwrite(rst_path, image)

if way==0:
    train()
else:
    rstImg_path = '/kaggle/working/pred_rst.jpg'
    # 进行单张图像推理  6,15,19,22,31,35,40,43,46,72,73,78,80,85,88,92
    rst = pred(image_path='./datasets/RFT/images/test/000006.jpg',rst_path=rstImg_path)
    if rst:
        print(f"保存成功：{rstImg_path}")
    else:
        print("出错")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

In [5]:
# 开始推理

# 所有相对路径基于/yolo8


# 0训练    1推理
way = 1


def train():
    from ultralytics import YOLO
    # Load a model
    model = YOLO("yolov8n.yaml")  # build a new model from scratch
    #model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

    # Use the model
    results = model.train(data="./datasets/RFT.yaml", epochs=1,imgsz=416,device=0)  # train the model

def pred(image_path,rst_path):
    from ultralytics import YOLO
    import cv2
    import yaml

    # 加载训练好的权重
    model = YOLO('./runs/detect/train/weights/best.pt')
    results = model.predict(source=image_path)

    # 读取原始图像
    image = cv2.imread(image_path)
    cv2.imwrite('/kaggle/working/pred_src.jpg',image)

    # 处理推理结果
    for result in results:
        boxes = result.boxes  # 获取边界框信息
        if boxes is not None:
            for box in boxes:
                # 获取边界框坐标
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                x1, y1, x2, y2 = xyxy

                # 获取类别索引和置信度
                cls = int(box.cls[0])
                conf = float(box.conf[0])

                with open('./datasets/RFT.yaml', 'r') as f:
                    data = yaml.safe_load(f)
                names = data['names']

                # 绘制边界框
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # 准备标签文本
                label = f'{names[cls]}: {conf:.2f}'

                # 绘制标签背景
                (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(image, (x1, y1 - label_height - 5), (x1 + label_width, y1), (0, 255, 0), -1)

                # 绘制标签文本
                cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # 保存带有检测结果的图像
    return cv2.imwrite(rst_path, image)

if way==0:
    train()
else:
    rstImg_path = '/kaggle/working/pred_rst.jpg'
    # 进行单张图像推理  6,15,19,22,31,35,40,43,46,72,73,78,80,85,88,92
    rst = pred(image_path='./datasets/RFT/images/test/000006.jpg',rst_path=rstImg_path)
    if rst:
        print(f"保存成功：{rstImg_path}")
    else:
        print("出错")

Ultralytics YOLOv8.0.29 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n summary (fused): 168 layers, 3007208 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /yolo8/datasets/RFT/images/test/000006.jpg: 416x416 5.9ms
Speed: 0.3ms pre-process, 5.9ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 416)


保存成功：/kaggle/working/pred_rst.jpg


In [6]:
# 训练结果转存
!zip -r /kaggle/working/train.zip /yolo8/runs/detect/train

  adding: yolo8/runs/detect/train/ (stored 0%)
  adding: yolo8/runs/detect/train/val_batch1_pred.jpg (deflated 7%)
  adding: yolo8/runs/detect/train/train_batch2.jpg (deflated 2%)
  adding: yolo8/runs/detect/train/val_batch2_pred.jpg (deflated 8%)
  adding: yolo8/runs/detect/train/val_batch2_labels.jpg (deflated 8%)
  adding: yolo8/runs/detect/train/P_curve.png (deflated 20%)
  adding: yolo8/runs/detect/train/weights/ (stored 0%)
  adding: yolo8/runs/detect/train/weights/best.pt (deflated 11%)
  adding: yolo8/runs/detect/train/weights/last.pt (deflated 11%)
  adding: yolo8/runs/detect/train/F1_curve.png (deflated 23%)
  adding: yolo8/runs/detect/train/results.csv (deflated 71%)
  adding: yolo8/runs/detect/train/val_batch1_labels.jpg (deflated 7%)
  adding: yolo8/runs/detect/train/args.yaml (deflated 51%)
  adding: yolo8/runs/detect/train/results.png (deflated 22%)
  adding: yolo8/runs/detect/train/R_curve.png (deflated 22%)
  adding: yolo8/runs/detect/train/val_batch0_pred.jpg (deflate